<a href="https://colab.research.google.com/github/Jaishree935/Image-Classification-using-CNN/blob/main/Image_Classification_using_Advanced_Hyper_Parameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Image Classification using CNN with the CIFAR-10 Dataset using Advanced HyperParameter Tunning**

# Steps to be followed
1. Importing necessary libraries
2. Loading and pre-processing of the CIFAR-10 dataset.
3. Defining the class names for CIFAR-10 images.
4. Define a function to build the model
5. Define the Tuner
6. Perform the Hyperparameter search
7. Get the best Hyperparameters
8. Build the model with the best Hyperparameters and train it
9. Plotting the training and validation accuracy and loss values

# Installation

In [ ]:
pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


# Step 1: Importing necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras_tuner.tuners import RandomSearch
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, models
import keras_tuner as kt

# Step 2: Loading and pre-processing of the CIFAR-10 dataset.

In [ ]:
(train_images, train_labels),(test_images, test_labels) = datasets.cifar10.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


# Step 3:  Defining the class names for CIFAR-10 images.

In [ ]:
class_name = ['airplane' , 'automobile' , 'bird' , 'cat' ,'deer' ,'dog' ,'frog' ,'horse','ship','truck']

# Step 4: Define a function to build the model


In [ ]:
def build_model(hp):
  model=models.Sequential()

  #Tune the number of Convolutional Layers(1,2 or 3)
  for i in range(hp.Int('conv_layers',1,3)):
    if i==0:
      model.add(layers.Conv2D(
          filters=hp.Int('filters_'+str(i) , min_value=32, max_value=128, step=16),
          kernel_size=(3,3),
          activation ='relu',
          input_shape=(32,32,3)
      ))
    else:
      model.add(layers.Conv2D(
          filters=hp.Int('filters_'+str(i) , min_value=32, max_value=128, step=16),
          kernel_size=(3,3),
          activation ='relu',
          padding='same'
      ))
      model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.Flatten())

#Tune the number of Dense Layers(1,2, or 3)
  for i in range(hp.Int('dense_layers',1,3)):
    model.add(layers.Dense(
        units=hp.Int('units_'+str(i), min_value=32, max_value=128, step=16),
        activation='relu'
    ))

    #Tune the dropout rate
    model.add(layers.Dropout(rate=hp.Float('dropout_' +str(i),min_value=0.0,max_value=0.5,step=0.1)))

  #The last dense layer with 10 output units(for 10 classes of CIFAR-10 dataset)
  model.add(layers.Dense(10,activation='softmax'))
  optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))

  model.compile(optimizer=optimizer,
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

  return model

# Step 5: Define the Tuner

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cifar10_tunning'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Step 6: Perform the Hyperparameter search

In [ ]:
tuner.search(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

# Step 7: Get the best Hyperparameters

In [ ]:
best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]

# Step 8 : Build the model with the best Hyperparameters and train it

In [ ]:
model=tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs=10,validation_data=(test_images,test_labels))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 190s 120ms/step - accuracy: 0.0992 - loss: 2.3138 - val_accuracy: 0.1000 - val_loss: 2.3037
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 199s 118ms/step - accuracy: 0.1028 - loss: 2.3038 - val_accuracy: 0.1000 - val_loss: 2.3035
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 180s 115ms/step - accuracy: 0.1006 - loss: 2.3041 - val_accuracy: 0.1000 - val_loss: 2.3040
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 180s 115ms/step - accuracy: 0.1016 - loss: 2.3040 - val_accuracy: 0.1000 - val_loss: 2.3040
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 202s 115ms/step - accuracy: 0.1015 - loss: 2.3040 - val_accuracy: 0.1000 - val_loss: 2.3034
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 203s 116ms/step - accuracy: 0.0982 - loss: 2.3040 - val_accuracy: 0.1000 - val_loss: 2.3039
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 200s 115ms/step - accuracy: 0.0995 - loss: 2.3038 - val_accuracy: 0.1000 - val_loss: 2.3050
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 179s 115ms/step - ac

# Step 9: Plotting the training and validation accuracy and loss values

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'],label='Training Accuracy')
plt.plot(history.history['val_accuracy'],label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim([0,1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.grid()


plt.subplot(1,2,2)
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim([0,1])
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid()

plt.show()
